<a href="https://colab.research.google.com/github/SolKidonakis/NLP2024/blob/main/TP2/TP2NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas necesarias
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install selenium
!pip install chromedriver_autoinstaller
!pip install networkx
!pip install matplotlib


In [ ]:
!pip install PyPDF2


In [ ]:
# Instalar dependencias necesarias si no están instaladas
!pip install selenium webdriver_manager beautifulsoup4


In [ ]:
!pip install pymupdf


In [ ]:
# Instalar las dependencias necesarias
!apt-get update # Actualizar el sistema
!apt install -y chromium-chromedriver  # Instalar ChromeDriver en Colab
!pip install selenium  # Instalar Selenium

In [ ]:
!apt-get install -y poppler-utils  # Instalar dependencias necesarias para pdf2image
!pip install pdf2image  # Instalar pdf2image
!pip install pytesseract  # Instalar pytesseract para OCR
!apt-get install -y tesseract-ocr  # Instalar Tesseract OCR


**LIBRERIAS**

In [140]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import fitz
from pdf2image import convert_from_path
import pytesseract
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

In [56]:
# Configuración de opciones de Selenium para usar un navegador sin cabeza (headless)
options = Options()
options.add_argument('--headless')  # Ejecutar en segundo plano
options.add_argument('--no-sandbox')  # Desactivar el sandbox
options.add_argument('--disable-dev-shm-usage')  # Solucionar problemas de recursos
options.add_argument('--remote-debugging-port=9222')  # Establecer puerto de depuración
options.add_argument('--disable-gpu')  # Deshabilitar la aceleración de GPU (útil en entornos virtuales)

# Configurar el driver con el servicio de ChromeDriver remoto
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

**WEB SCRAPPING PARA DOCUMENTOS DE TEXTO**

**MANUAL DE JUEGO**

In [65]:
# Iniciar el navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL del juego "Viticulture" en BoardGameGeek
url = 'https://boardgamegeek.com/boardgame/128621/viticulture'

# Navegar a la página
driver.get(url)

# Esperar unos segundos para que la página cargue completamente
time.sleep(5)

# Navegar a la sección "Files"
files_section = driver.find_element(By.LINK_TEXT, 'Files')
files_section.click()

# Esperar que la sección de archivos cargue
time.sleep(5)

# Buscar el enlace al archivo PDF del "Final Rulebook for the Second Printing"
file_link = driver.find_element(By.PARTIAL_LINK_TEXT, "Final Rulebook for the Second Printing")

# Obtener el enlace completo
pdf_url = file_link.get_attribute('href')
print("Enlace al manual PDF:", pdf_url)

# Cerrar el navegador al terminar
driver.quit()



Enlace al manual PDF: https://boardgamegeek.com/filepage/99700/final-rulebook-for-the-second-printing


In [66]:
# Descargar el PDF desde el enlace
response = requests.get(pdf_url)

# Guardar el archivo PDF en el sistema local
pdf_filename = "/mnt/data/viticulture_rulebook.pdf"
with open(pdf_filename, 'wb') as f:
    f.write(response.content)

print(f"Archivo PDF descargado en: {pdf_filename}")

# Cerrar el navegador al terminar
driver.quit()

Archivo PDF descargado en: /mnt/data/viticulture_rulebook.pdf


In [75]:


# Ruta al archivo PDF descargado
pdf_filename = "/mnt/data/viticulture_rulebook.pdf"

# Convertir el PDF a imágenes (una imagen por página)
pages = convert_from_path(pdf_filename, 300)  # 300 DPI para buena calidad de imagen

# Extraer texto de las imágenes usando OCR
pdf_text = ""
for i, page in enumerate(pages):
    # Convertir la página a texto usando Tesseract
    page_text = pytesseract.image_to_string(page)
    pdf_text += page_text
    print(f"Texto extraído de la página {i + 1}:")
    print(page_text[:1000])  # Mostrar solo los primeros 1000 caracteres

# Mostrar el texto extraído completo (las primeras 2000 líneas para evitar demasiada salida)
print("Texto extraído completo:")
print(pdf_text[:2000])  # Muestra los primeros 2000 caracteres


Texto extraído de la página 1:
 

SE __

  

ea ee

=

Sel

Old-world Tuscany awaits your winemaking skill and Strategic cunning.
You'll plant vines, harvest grapes, age wines, and fill merchant orders to create the greatest winery in [taly!

2 — 6 PLAYERS AGES 13 & UP * 45 — 90 MINUTES

VITICULTURE IS A WORKER-PLACEMENT STRATEGY GAME THAT
ALLOWS PLAYERS TO CREATE THEIR OWN TUSCAN VINEYARD
ANYWHERE A TABLE AND A FRIEND CAN BE FOUND.

OBJECT OF THE GAME

Players begin the game with an old crush pad, three empty fields,

a small cellar, two workers, and one grande worker. Players allocate
workers and the occasional visitor to complete various tasks
throughout the year. In the spring, players decide the schedule their
workers will follow. An early rise gives the player the chance to act first;
however, offering workers a later wake-up time gives you escalating
bonuses. In the summer, players place workers on action spaces

to plant vines and build structures that help advance that player'

**RESEÑAS**

In [84]:
headers = {
    'User-Agent': 'Mozilla/5.0',
}

crawl_delay = 5  # Pausa de 5 segundos entre solicitudes


In [85]:
url_resena = 'https://misutmeeple.com/2015/03/resena-viticulture/'

response = requests.get(url_resena, headers=headers)
time.sleep(crawl_delay)


In [87]:
soup = BeautifulSoup(response.content, 'html.parser')
contenido_resena = soup.find('div', class_='entry-content single-content')
# Lista de etiquetas que contienen contenido relevante
etiquetas_contenido = ['p', 'h2', 'h3', 'ul', 'ol']

elementos = contenido_resena.find_all(etiquetas_contenido)

# Extraer el texto y limpiar
texto_resena = ''
for elemento in elementos:
    texto = elemento.get_text(separator=' ', strip=True)
    texto_resena += texto + '\n\n'


In [ ]:
with open('viticulture_resena.txt', 'w', encoding='utf-8') as f:
    f.write(texto_resena)


In [88]:
def limpiar_texto(texto):
    # Eliminar espacios múltiples y caracteres especiales
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ.,;:!?-]', '', texto)
    return texto.strip()

texto_resena_limpio = limpiar_texto(texto_resena)

# Sobrescribir el archivo con el texto limpio
with open('viticulture_resena.txt', 'w', encoding='utf-8') as f:
    f.write(texto_resena_limpio)


In [89]:
def contar_palabras(texto):
    return len(texto.split())

total_palabras_resena = contar_palabras(texto_resena_limpio)
numero_paginas_resena = total_palabras_resena / 500  # Asumiendo 500 palabras por página

print(f'Total de palabras en la reseña: {total_palabras_resena}')
print(f'Número estimado de páginas: {numero_paginas_resena}')


Total de palabras en la reseña: 5603
Número estimado de páginas: 11.206


In [90]:
# Leer el contenido del archivo
with open('viticulture_resena.txt', 'r', encoding='utf-8') as archivo:
    contenido_resena = archivo.read()

# Mostrar el contenido
print(contenido_resena)


Introducción La Toscana espera tu habilidad para elaborar vino y tu astucia estratégica. Plantarás vides, cosecharás la uva, envejecerás los vinos y cumplirás los pedidos de los comerciantes para crear la mejor bodega de Italia. Y así se nos presenta Viticulture , un juego diseñado por diseñado por Jamey Stegmaier y Alan Stone, los también responsables de Euphoria: Build a Better Dystopia. El juego fue financiado mediante una campaña de micromecenazgo a través de KickStarter en 2013. Para tal efecto se fundó la editorial Stonemaier Games fusión de parte de los apellidos de los dos autores. El pasado 2014 tuvo una segunda edición con motivo del lanzamiento de Tuscany, la expansión de Viticulture. De las ilustraciones se encargaron Jacqui Davis, Beth Sobel y David Montgomery. Los dos primeros ilustradores también dieron vida a Euphoria, mientras que el sr. Montgomery no tiene otros proyectos en el mundo lúdico a parte del título que nos ocupa. El juego no ha sido editado en nuestro país 

**FOROS**

In [150]:
# Configuración de opciones de Selenium para usar un navegador sin cabeza (headless)
options = Options()
options.add_argument('--headless')  # Ejecutar en segundo plano
options.add_argument('--no-sandbox')  # Desactivar el sandbox
options.add_argument('--disable-dev-shm-usage')  # Solucionar problemas de recursos
options.add_argument('--remote-debugging-port=9222')  # Establecer puerto de depuración
options.add_argument('--disable-gpu')  # Deshabilitar la aceleración de GPU (útil en entornos virtuales)
options.add_argument('--disable-software-rasterizer')  # Asegura que Chrome funcione bien en entornos virtuales

# Configurar el driver con el servicio de ChromeDriver remoto
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [151]:

try:
    # URL de la página de foros
    base_url = 'https://boardgamegeek.com/boardgame/128621/viticulture/forums/0'

    # Navegar a la página de foros
    driver.get(base_url)

    # Esperar unos segundos para que la página cargue completamente
    time.sleep(5)

    # Lista para almacenar todas las URLs de los hilos
    all_forum_urls = []
    unique_urls = set()

    # Número de páginas a procesar
    max_pages = 1  # Solo la primera página
    current_page = 1

    while current_page <= max_pages:
        print(f"Extrayendo datos de la página {current_page}...")

        try:
            # Esperar hasta que los enlaces de los hilos estén presentes
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//a[starts-with(@href, '/thread/') and not(contains(@href, '/thread/new'))]")
                )
            )
            print("Hilos encontrados, continuando con la extracción...")
        except TimeoutException:
            print(f"Timeout al esperar los hilos en la página {current_page}.")
            break

        # Obtener los enlaces correctos de los hilos de discusión usando XPath
        try:
            forum_links = driver.find_elements(
                By.XPATH, "//a[starts-with(@href, '/thread/') and not(contains(@href, '/thread/new'))]"
            )
            print(f"Total de enlaces encontrados: {len(forum_links)}")
        except NoSuchElementException:
            print(f"No se encontraron hilos en la página {current_page}.")
            break

        # Iterar sobre cada enlace de hilo
        for link in forum_links:
            try:
                forum_url = link.get_attribute('href')
                title = link.text.strip()

                # Filtrar títulos vacíos o que no contengan letras
                if forum_url and title and re.search('[a-zA-Z]', title):
                    # Validar que el href siga el patrón '/thread/{número}/...'
                    if re.match(r'^https?://boardgamegeek\.com/thread/\d+/', forum_url):
                        if forum_url not in unique_urls:
                            unique_urls.add(forum_url)
                            all_forum_urls.append({'title': title, 'url': forum_url})
                            print(f"--- Título: {title} ---")
                            print(f"URL del foro: {forum_url}")
            except StaleElementReferenceException:
                # Si el elemento ya no está en el DOM, omitir
                continue

        print(f"Total de hilos válidos extraídos en esta página: {len(all_forum_urls)}")

        # Como max_pages = 1, no navegaremos a la siguiente página
        break

    # Ahora, all_forum_urls contiene todas las URLs de los hilos extraídos de la primera página
    print(f"Total de hilos extraídos en total: {len(all_forum_urls)}")

    if not all_forum_urls:
        print("No se encontraron hilos válidos para extraer.")
    else:
        # Lista para almacenar los datos extraídos
        data = []

        # Iterar sobre cada URL y extraer el contenido de los hilos
        for idx, forum in enumerate(all_forum_urls, start=1):
            forum_url = forum['url']
            title = forum['title']
            print(f"Extrayendo datos del hilo {idx}...")
            print(f"--- Título: {title} ---")
            print(f"URL del foro: {forum_url}")

            try:
                # Navegar al hilo de discusión para extraer su contenido
                driver.get(forum_url)

                # Esperar hasta que el contenido del hilo esté visible y completamente cargado
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.post-body'))
                )
            except TimeoutException:
                print(f"No se pudo esperar correctamente el contenido del hilo: {forum_url}.")
                print("\n" + "-"*80 + "\n")
                continue
            except Exception as e:
                print(f"Error al cargar el hilo: {forum_url}. Error: {e}")
                print("\n" + "-"*80 + "\n")
                continue

            # Obtener el contenido de la página con BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Extraer todos los posts en el hilo
            posts = soup.select('div.post-body')

            post_contents = []
            if posts:
                for post_num, post in enumerate(posts, start=1):
                    # Seleccionar el contenido del mensaje
                    post_text = post.find('gg-markup-safe-html')
                    if post_text:
                        content = post_text.get_text(separator='\n', strip=True)
                        post_contents.append(content)
                        print(f"Contenido del post {post_num}: {content[:500]}...")  # Mostrar solo los primeros 500 caracteres
                    else:
                        print(f"No se encontró el contenido del post {post_num}.")
            else:
                post_contents.append("No se encontró el contenido de los posts.")
                print("No se encontró el contenido de los posts.")

            # Agregar los datos a la lista
            data.append({
                'Título': title,
                'URL': forum_url,
                'Contenido': "\n\n".join(post_contents)
            })

            print("\n" + "-"*80 + "\n")
            # Opcional: Pausa entre hilos para evitar sobrecargar el servidor
            time.sleep(2)

        # Guardar los datos en un archivo TXT
        with open('foros_viticulture.txt', 'w', encoding='utf-8') as txtfile:
            for entry in data:
                txtfile.write(f"Título: {entry['Título']}\n")
                txtfile.write(f"URL: {entry['URL']}\n")
                txtfile.write("Contenido:\n")
                txtfile.write(f"{entry['Contenido']}\n")
                txtfile.write("="*80 + "\n\n")

        print("Datos guardados en 'foros_viticulture.txt'")

finally:
    # Cerrar el navegador al terminar, incluso si ocurre un error
    driver.quit()

Extrayendo datos de la página 1...
Hilos encontrados, continuando con la extracción...
Total de enlaces encontrados: 157
--- Título: Adding up wine. ---
URL del foro: https://boardgamegeek.com/thread/3417770/adding-up-wine
--- Título: When is trained worked able to be used if trained after already left winter? ---
URL del foro: https://boardgamegeek.com/thread/3356000/when-is-trained-worked-able-to-be-used-if-trained
--- Título: Special Worker Cards ---
URL del foro: https://boardgamegeek.com/thread/3334618/special-worker-cards
--- Título: Could you win by choosing the wake up position #6 for the victory point every year? ---
URL del foro: https://boardgamegeek.com/thread/3333938/could-you-win-by-choosing-the-wake-up-position-6-f
--- Título: IS it viable to sell and buy the same field each year using the bonus space for the victory point? ---
URL del foro: https://boardgamegeek.com/thread/3333936/is-it-viable-to-sell-and-buy-the-same-field-each-y
--- Título: Making 7, 8, 9 value red/wh